<a href="https://colab.research.google.com/github/Oukey/M_L/blob/master/myML1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Занятие 5. Классификация изображений**

https://vk.com/lambda_brain

Классификация изображений -- это уже полноценная задача реального мира. Самое приятное здесь в том, что нету качественного алгоритмического отличия классификации изображений от классификации других наборов данных. Нейронные сети работают таким магическим образом, что мы просто подаём им на вход наборы "байтов", и не важно, что это -- абстрактные значения функций или пикселы изображений, они все анализируются нейросетью по единой схеме.

Мы познакомимся с основными принципами классификации изображений в PyTorch, а также изучим смежные вопросы, связанные с загрузкой объёмных датасетов с изображениями в программу и их удобной визуализацией.


# New Section



---


В качестве первого примера воспользуемся распространённым датасетом изображений **MNIST**, на котором создано наверное максимальное количество обучающих уроков по нейронным сетям.

MNIST подготовлен **Яном ЛеКуном** -- ведущим мировым специалистом по нейронным сетям, и представляет собой датасет для обучения распознаванию рукописных цифр. Обучающая выборка содержит 60,000 изображений, и тестовая -- 10,000. Каждое изображение чёрно-белое размером 28x28 пикселов.

Импортируем нужные модули:


In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms


Задаём гипер-параметры:

In [0]:

input_size = 28*28     # Размер изображения в точках
hidden_size = 500      # Количество нейронов в скрытом слое
num_classes = 10       # Количество распознающихся классов (10 цифр)
n_epochs = 2           # Количество эпох
batch_size = 4         # Размер мини-пакета входных данных
lr = 0.01              # Скорость обучения

Большое количество готовых датасетов и различные удобные методы их загрузки и обработки собраны в модуле torchvision. Имеется в нём конечно и MNIST.

В конструкторе датасета MNIST указывается, как правило, каталог root, где будет локально размещён соответствующий датасет, download -- необходимость скачивания датасета для его локального использования, train -- признак, является ли данный датасет обучающим или тестовым, и трансформация transform, которую надо выполнить над данным датасетом. Последняя фича весьма полезна, потому что исходные изображения нередко требуется предварительно обрабатывать под конкретный формат их дальнейшего анализа. В нашем случае просто укажем стандартную трансформацию преобразования входных данных в тензоры.

Убедимся, что размеры датасета совпадают с официально объявленными.


In [0]:
import torchvision.transforms as transforms
mnist_trainset = dsets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_testset = dsets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
print(len(mnist_trainset))
print(len(mnist_testset))

Так как наш датасет большой по размеру, обрабатывать его надо небольшими порциями, мини-пакетами, как рассказывалось во втором занятии -- с помощью загрузчиков данных DataLoader.

In [0]:
train_loader = torch.utils.data.DataLoader(dataset=mnist_trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True) # загрузчик обучающих данных
test_loader = torch.utils.data.DataLoader(dataset=mnist_testset, 
                                          batch_size=batch_size, 
                                          shuffle=False) # загрузчик тестовых данных

Добавим наш стандартный шаг обучения.

In [0]:
# импортируем нужные библиотеки
import torch
import numpy as np # всегда пригодится :)
from torch.nn import Linear, Sigmoid

# инициализируем девайс
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# добавляем типовую функцию "шаг обучения"
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

Структура нашей модели будет представлять собой двуслойную нейронную сеть -- она ничем не отличается от модели, применявшейся в прошлом занятии при анализе абстрактного датасета (линейный вход + ReLU + линейный выход).

Воспользуемся лоссом CrossEntropyLoss() и методом оптимизации Adam.

Обратите внимание, что мы задаём уже не тысячи, а **всего две эпохи** -- процесс обучения на таком довольно объёмном датасете потребует уже прилично времени (несколько минут на одну эпоху).

Единственное дополнение -- мы применяем метод reshape(-1, 28 * 28), чтобы изменить входной трёхмерный формат изображения 1x28x28 (глубина цветности x размеры) на нужный нам одномерный вектор длиной 784 значения. Параметр -1 означает, что преобразование выполняется в одномерный результат.


In [0]:
from torch import optim, nn

model = torch.nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, num_classes))
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
  
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for images, labels in train_loader:
        images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        loss = train_step(images, labels)
    print(epoch)

print(model.state_dict())    
print(loss)

Теперь проверим, с какой точностью наша модель проверит 10,000 тестовых изображений:

In [0]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Точность: {} %'.format(100 * correct / total))

Около 90% -- это очень хороший результат для тренировки всего в течение пяти минут за две эпохи. Обратите внимание, что обученная модель время на распознавание даже такого приличного по объёму датасета уже практически не тратит.


---



Вычисление, конечно, может занимать не минуты, а часы, сутки, недели и месяцы работы. Чтобы продолжить работу с нашей обученной моделью уже в прикладных задачах, нам надо прежде всего её сохранить в файле (в PyTorch принято, что расширение файла с полностью сохранённой моделью -- .pt).


In [0]:
torch.save(model, 'mnist_full.pt')

Сохранение выполняется в виртуальной машине гугла, где мы загрузили датасет MNIST. Загрузка модели выполняется так же просто:

In [0]:
model = torch.load("mnist_full.pt")
model.eval()

После загрузки модели надо обязательно выполнить её метод **eval()** -- это означает, что мы будем пользоваться моделью в режиме её эксплуатации, а не в режимах обучения или каких-то других. 



---


Наконец, в заключение проверим результаты работы нашей обученной модели визуально. Запомните этот типовой код, его удобно применять в самых разных задачах классификации изображений. Он выводит четыре случайных изображения из датасета и указывает под ними соответствующие им классы (прогноз), которые заданы в списке classes.

In [0]:
import matplotlib.pyplot as plt
import numpy as np

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(' '.join('%10s' % classes[labels[j]] for j in range(4)))

##**Задание**

Попробуйте повысить точность нашей модели:

* увеличьте количество эпох;
* увеличьте количество нейронов;
* добавьте новые скрытые слои;
* измените размер мини-пакета.


---





Далее мы рассмотрим работу с более сложным датасетом изображений (котики и пёсики), которому потребуется предварительная обработка.